In [1]:
import os
import sys 
from pathlib import Path


In [2]:
project_root = Path(__file__).parent.parent if "__file__" in locals() else Path().resolve().parent
sys.path.append(str(project_root))

In [11]:
from dataclasses import dataclass

from src.Briefly.constants import * 
from src.Briefly.utils.common import read_yaml, create_dirs

from src.Briefly.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

In [9]:
@dataclass
class DataTransformConfig:
    root_dir: Path
    dataset_path: Path
    tokenizer: Path

Configuration:

In [10]:
class ConfigManager: 
    def __init__(self, config_path: Path = CONFIG_PATH, params_path: Path = PARAMS_PATH):
        self.config = read_yaml(config_path) 
        self.params = read_yaml(params_path)

        create_dirs([self.config.artifacts_root])
    
    def get_datatransform_config(self) -> DataTransformConfig:
        config = self.config.data_transform

        create_dirs([config.root_dir])

        data_transform_config = DataTransformConfig(
            root_dir=config.root_dir,
            dataset_path=config.dataset_path,
            tokenizer=config.tokenizer,
        )
        return data_transform_config


Components:

In [ ]:
class DataTransform:
    def __init__(self, config:DataTransformConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer)
    def features_maker(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }